In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

# kd-tree构造算法

In [1]:
def kdtree( data, divide_depth=1 ):
    """
    build a kd-tree for O(n log n) nearest neighbour search

    input:
        data:       2D ndarray, shape =(ndim,ndata), preferentially C order
        leafsize:   max. number of data points to leave in a leaf

    output:
        kd-tree:    list of tuples
    """

    ndim = data.shape[0]
    ndata = data.shape[1]

    # find bounding hyper-rectangle
    hrect = np.zeros((2,data.shape[0]))
    hrect[0,:] = data.min(axis=1)
    hrect[1,:] = data.max(axis=1)

    # create root of kd-tree
    idx = np.argsort(data[0,:], kind='mergesort')
    data[:,:] = data[:,idx]
    splitval = data[0,ndata/2]

    left_hrect = hrect.copy()
    right_hrect = hrect.copy()
    left_hrect[1, 0] = splitval
    right_hrect[0, 0] = splitval

    # idx, data, left_hrect, right_hrect, left_nodeptr, right_nodeptr
    tree = [(None, None, left_hrect, right_hrect, None, None)]
    # data, idx, depth, parent, leftbranch
    stack = [(data[:,:ndata/2], idx[:ndata/2], 1, 0, True),
             (data[:,ndata/2:], idx[ndata/2:], 1, 0, False)]

    # recursively split data in halves using hyper-rectangles:
    while stack:

        # pop data off stack
        data, didx, depth, parent, leftbranch = stack.pop()
        ndata = data.shape[1]
        nodeptr = len(tree)

        # update parent node

        _didx, _data, _left_hrect, _right_hrect, left, right = tree[parent]

        tree[parent] = (_didx, _data, _left_hrect, _right_hrect, nodeptr, right) if leftbranch \
            else (_didx, _data, _left_hrect, _right_hrect, left, nodeptr)

        # insert node in kd-tree

        # leaf node?
        if depth <= divide_depth:
        # if ndata <= leafsize:
            _didx = didx.copy()
            _data = data.copy()
            # leaf = (_didx, _data, None, None, 0, 0)
            leaf = (_didx, _data, None, None, -1, -1)
            tree.append(leaf)

        # not a leaf, split the data in two      
        else:
            splitdim = depth % ndim
            idx = np.argsort(data[splitdim,:], kind='mergesort')
            data[:,:] = data[:,idx]
            didx = didx[idx]
            nodeptr = len(tree)
            stack.append((data[:,:ndata/2], didx[:ndata/2], depth+1, nodeptr, True))
            stack.append((data[:,ndata/2:], didx[ndata/2:], depth+1, nodeptr, False))
            splitval = data[splitdim,ndata/2]
            if leftbranch:
                left_hrect = _left_hrect.copy()
                right_hrect = _left_hrect.copy()
            else:
                left_hrect = _right_hrect.copy()
                right_hrect = _right_hrect.copy()
            left_hrect[1, splitdim] = splitval
            right_hrect[0, splitdim] = splitval
            # append node to tree
            tree.append((None, None, left_hrect, right_hrect, None, None))

    return tree

# 分块提取特定时间的出租车客流数据

In [ ]:
size = 10 ** 6
start_time = pd.to_datetime('20140824 7:0:0')
end_time = pd.to_datetime('20140824 8:0:0')
# start_time = '20140824 7'
# end_time = '20140824 8'
hour_dataframe = pd.DataFrame()
for chunk in pd.read_csv('20140824_train.txt', chunksize=size):
    chunk.columns = ['taxi_id', 'latitude', 'longitude', 'passenger', 'time']
    chunk.time = chunk.time.apply(pd.to_datetime)
    mask = (chunk.time >= start_time) & (chunk.time < end_time)
    hour_dataframe = pd.concat([hour_dataframe, chunk.loc[mask]])

# 对每辆出租车在特定时间内的上客、下客点进行统计。

In [ ]:
up_off_pairs = []
grouped = hour_dataframe.groupby('taxi_id', sort=False)
for taxi_id, group in grouped:
    selected_group_length = len(group)
    group = group.sort_values(by=['time'])
    i = 0
    while i < selected_group_length - 1:
        if ((group.iloc[[i]].passenger == 1).bool() & (group.iloc[[i + 1]].passenger != 0).bool()):
            up_point_row = group.iloc[[i]]
            up_off_point = [[up_point_row.longitude, up_point_row.latitude]]
            j = i + 1
            while j < selected_group_length:
                print('current_j is %s' % j)
                if (group.iloc[[j]].passenger == 0).bool():
                    off_point_row = group.iloc[[j - 1]]
                    up_off_point.append([off_point_row.longitude, off_point_row.latitude])
                    i = j + 1
                    break
                else:
                    if j == selected_group_length - 1:
                        up_off_point.append([group.iloc[[j]].longitude, group.iloc[[j]].latitude])
                        i = j
                        break
                    else:
                        j += 1
            if len(up_off_point) == 2:
                up_off_pairs.append(up_off_point)
        else:
            i += 1
print(up_off_pairs)